In [3]:
import pandas as pd

ad_clicks = pd.read_csv('ad_clicks.csv')
print(ad_clicks.head())




                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


In [4]:
#number of views per source
views_per_source = ad_clicks.groupby('utm_source').user_id.count().reset_index()
print(views_per_source)



  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


In [5]:
#add column if the video was click (timestamp not null true or false)
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
print(ad_clicks.head())


                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  
3                NaN                  B     False  
4                NaN                  B     False  


In [6]:
#the percentage of people who clicked on the add by source
#start by gourping by source and if clicked
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
print(clicks_by_source)

clicks_pivot = clicks_by_source.pivot(columns='is_click', index='utm_source', values='user_id').reset_index()
print(clicks_pivot)



  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66
is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


In [7]:
#percentage of clicked views by source
clicks_pivot['percent_clicked'] = clicks_pivot[True]/(clicks_pivot[False] + clicks_pivot[True])
print(clicks_pivot)



is_click utm_source  False  True  percent_clicked
0             email    175    80         0.313725
1          facebook    324   180         0.357143
2            google    441   239         0.351471
3           twitter    149    66         0.306977


In [8]:
#number in experiment group (A/B)
experiment_groups = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
print(experiment_groups)

# experiment group by is_clicked
experiment_groups_clicked = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()
print(experiment_groups_clicked)
#results in a pivot table
groups_pivot = experiment_groups_clicked.pivot(columns='is_click', index='experimental_group', values='user_id').reset_index()
print(groups_pivot)



  experimental_group  user_id
0                  A      827
1                  B      827
  experimental_group  is_click  user_id
0                  A     False      517
1                  A      True      310
2                  B     False      572
3                  B      True      255
is_click experimental_group  False  True
0                         A    517   310
1                         B    572   255


In [9]:
#dataframes for each expereiment group
a_clicks = ad_clicks[ad_clicks.experimental_group == "A"]
b_clicks = ad_clicks[ad_clicks.experimental_group == "B"]

#group each by day 
a_clicks_day = a_clicks.groupby('day').user_id.count().reset_index()
print(a_clicks_day)
b_clicks_day = b_clicks.groupby('day').user_id.count().reset_index()
print(b_clicks_day)



             day  user_id
0     1 - Monday      113
1    2 - Tuesday      119
2  3 - Wednesday      124
3   4 - Thursday      116
4     5 - Friday      128
5   6 - Saturday      118
6     7 - Sunday      109
             day  user_id
0     1 - Monday      113
1    2 - Tuesday      119
2  3 - Wednesday      124
3   4 - Thursday      116
4     5 - Friday      128
5   6 - Saturday      118
6     7 - Sunday      109


In [10]:
#percentage by day
a_clicks_day["day_percentage"] = round(a_clicks_day.user_id / a_clicks_day.user_id.sum() * 100, 1)
print(a_clicks_day)
b_clicks_day["day_percentage"] = round(b_clicks_day.user_id / b_clicks_day.user_id.sum() * 100, 1)
print(b_clicks_day)

             day  user_id  day_percentage
0     1 - Monday      113            13.7
1    2 - Tuesday      119            14.4
2  3 - Wednesday      124            15.0
3   4 - Thursday      116            14.0
4     5 - Friday      128            15.5
5   6 - Saturday      118            14.3
6     7 - Sunday      109            13.2
             day  user_id  day_percentage
0     1 - Monday      113            13.7
1    2 - Tuesday      119            14.4
2  3 - Wednesday      124            15.0
3   4 - Thursday      116            14.0
4     5 - Friday      128            15.5
5   6 - Saturday      118            14.3
6     7 - Sunday      109            13.2
